In [3]:
#Preamble. Standard packages for to load
import astropy
from astropy.table import Table, Column, MaskedColumn, vstack 
import numpy as np
import matplotlib.pyplot as plt
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 
from astropy.io import fits
import astropy.io.ascii as ascii
from astropy.modeling import models, fitting
from numpy import ones
from numpy.linalg import lstsq
import pandas as pd

In [4]:
good_WD = Table.read('DA_with_LP_Mike.dat',format='ascii')
lambda_balmer = np.array([6563, 4861, 4341, 4102, 3970, 3889])

In [5]:
# Our model has a temperature effect, then absorption features on top of the model
from numpy import vstack
# Constants
h_planck = 4.136e-15 # in eV*s
c_light = 2.998e10   # in cgs
k_boltz = 8.617e-5   # in eV/K

def my_line(points, x_vec):
    x_coords, y_coords = zip(*points)
    A = vstack([x_coords,ones(len(x_coords))]).T
    m, c = lstsq(A, y_coords)[0]
    return m*x_vec + c, m, c

def FWHM(height,norm):
    gam = 1/(np.pi*np.abs(height))
    return gam*norm
new_centers = np.zeros(6)
iteration = 0

In [6]:
slope_thresholds = [1,2,3,4,5,6]
shift_thresholds = [0,0,0,0,50,50]

In [7]:
# #windows for line (on either side): 120, 120, 120, 60, 50, 40
# #Feed all six models in and fit at once.
# # Balmer line centers (in cgs):
def obtain_lorentz_parameters(catalog,iteration):
    count = 0
    lambda_balmer = np.array([6563, 4861, 4341, 4102, 3970, 3889])
    slope_thresholds = [19,55,91,130,200,240]
    shift_thresholds = [0,0,0,0,50,50]
    #120 either side for H_alpha,beta,gamma;60,50,40 
    #Main loop
    C_coeff_ls = np.zeros(6)
    x_center_ls = np.zeros(6)
    gamma_ls = np.zeros(6)
    total = 1
    for index,i in enumerate(catalog):
        #Gets data from file
        directory = "../data/"
        filename = directory+catalog['file'][index]

        try:
            data = fits.getdata(filename, 1)
        except:
            print("Missing", filename)
            continue

        all_flux = data['flux']
        all_lambda = 10 ** data['loglam']

        total = total + 1
        if total%100 == 0:  print(total)
        if iteration != 0:
            lambda_balmer = new_centers

        for ind,line in enumerate(lambda_balmer):
            ctitle = 'C'+str(iteration)+'_'+str(ind)
            xtitle = 'X'+str(iteration)+'_'+str(ind)
            gtitle = 'G'+str(iteration)+'_'+str(ind)
            mtitle = 'M'+str(iteration)+'_'+str(ind)
            ytitle = 'Y'+str(iteration)+'_'+str(ind)
            chititle = 'Chi'+str(iteration)+'_'+str(ind)
            
            #Make the windows for lorentzian fitting
            window = np.where(np.logical_and(all_lambda>= (line - 15), all_lambda<= (line + 15)))[0]
            #Get an educated guess for C_true
            if not all_flux[window].size:
                continue
            thirds = int(len(all_flux[window])/3)
            left_max = np.max(all_flux[window][:thirds])
            right_max = np.max(all_flux[window][-thirds:])
            left_point = all_lambda[window][:thirds][np.argmax(all_flux[window][:thirds])]
            right_point = all_lambda[window][2*thirds + np.argmax(all_flux[window][-thirds:])]

            points = [(left_point,left_max),(right_point,right_max)]
            vert_fix = my_line(points,all_lambda[window])
   
            int_y = all_flux[window]-vert_fix[0]
            fixed_y = int_y*-1
            area = np.trapz(int_y)
            ampl = np.min(int_y)

            left_mean = np.mean(all_flux[window][:thirds])
            right_mean = np.mean(all_flux[window][-thirds:])
            left_lambda = np.mean(all_lambda[window][:thirds])
            right_lambda = np.mean(all_lambda[window][-thirds:])

            points = [(left_lambda,left_mean),(right_lambda,right_mean)]
            vert_shift = my_line(points,all_lambda[window])

            #Define initial guesses
            C_true = area*-1
            a_true = ampl*-1
            x_true = (all_lambda[window][0] + all_lambda[window[-1]])/2
            gamma_true = FWHM(np.min(all_flux[window]-vert_shift[0]),C_true)
            #print gamma_true

            result = models.Lorentz1D(amplitude = C_true, x_0 = x_true, fwhm = gamma_true)
            fit_ls = fitting.LevMarLSQFitter()
            g = fit_ls(result, all_lambda[window], fixed_y)
            chi_squ = np.sum((all_flux[window] - (-1*g(all_lambda[window])+vert_fix[0]))**2/(all_flux[window]))

            if chi_squ > .3*np.mean(all_flux[window]) or chi_squ > .4*good_WD['S_N'][index]:
#                 plt.figure(figsize=(8,5))
#                 plt.plot(all_lambda[window], all_flux[window], 'ko')
#                 plt.plot(all_lambda[window], -1*g(all_lambda[window]) + vert_fix[0], label='Model')
#                 plt.xlabel('Lambda')
#                 plt.ylabel('Flux')
#                 plt.title('Chi: %f  S_N: %f' %(chi_squ,good_WD['S_N'][index]))
#                 plt.legend(loc=2)
#                 plt.savefig('%f_%f.png' %(index,line))
#                 print chi_squ, good_WD['S_N'][index]
                count += 1

            new_centers[ind] = g.x_0.value
            catalog[ctitle][index] = g.amplitude.value
            catalog[xtitle][index] = g.x_0.value
            catalog[gtitle][index] = g.fwhm.value
            catalog[ytitle][index] = vert_shift[2]
            catalog[mtitle][index] = vert_shift[1]
            catalog[chititle][index] = chi_squ
            
            
#             catalog[ctitle][index] = C_coeff_ls[ind]
#             catalog[xtitle][index] = x_center_ls[ind]
#             catalog[gtitle][index] = gamma_ls[ind]
    print count

In [8]:
obtain_lorentz_parameters(good_WD,0)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600


MC1TEMDN=-0.00000000000000E+00 / sp1 mech median temp                            [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1THT  =-0.00000000000000E+00 / sp1 mech Temp_Hartmann_Top                      [astropy.io.fits.card]


1700
1800
1900
2000
2100
2200
2300
2400


MC1TRCT =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Top                       [astropy.io.fits.card]


2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400


MC1TRCB =-0.00000000000000E+00 / sp1 mech Temp_Red_Cam_Bot                       [astropy.io.fits.card]


4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
15431


In [9]:
good_WD.write('DA_with_LP_Mike.dat', format='ascii')

In [4]:
good_WD = good_WD.to_pandas()

In [5]:
good_WD = good_WD[good_WD.Chi0_4 != '--']
good_WD = good_WD[good_WD.Chi0_5 != '--']
good_WD['Chi0_4'] = pd.to_numeric(good_WD['Chi0_4'])
good_WD['Chi0_5'] = pd.to_numeric(good_WD['Chi0_5'])

In [6]:
def get_filename(plate,mjd,fiber,wd):
    try:
        plwd = wd[wd['Plate'] == plate]
        if len(plwd) == 0: raise Exception()
    except Exception:
        print 'No plate number'
        return ''
    try:
        mjwd = plwd[plwd['MJD'] == mjd]
        if len(mjwd) == 0: raise Exception()
    except Exception:
        print 'No mjd date'
        return ''
    try:
        fbwd = mjwd[mjwd['Fiber'] == fiber]
        if len(fbwd) == 0: raise Exception()
    except Exception:
        print 'No fiber number'
        return ''
    name = fbwd['file']
    return str(name[0])

In [8]:
def plot_spec(plate,mjd,fiber,wd):
    fits_spec = fits.open('../data/'+get_filename(plate,mjd,fiber,wd))
    wavelength = 10**fits_spec[1].data['loglam']
    flux = fits_spec[1].data['flux']
    fig, ax = plt.subplots(1, 2, figsize=(12,4))
    ax[0].plot(wavelength, flux)
    ax[1].plot(wavelength, flux)
    ax[1].set_xlim(3800, 4400)
    plt.show()

In [9]:
plot_spec(3760,55268,863,good_WD)

KeyError: 0

In [66]:
from astroML.plotting import scatter_contour


In [ ]:
plt.hist2d(good_WD['Chi0_0'],good_WD['S_N'],bins=200);

# plt.hist2d(good_WD['Chi0_1'],good_WD['S_N'],bins=200);

# plt.hist2d(good_WD['Chi0_2'],good_WD['S_N'],bins=200);

# plt.hist2d(good_WD['Chi0_3'],good_WD['S_N'],bins=200);

# plt.hist2d(good_WD['Chi0_4'],good_WD['S_N'],bins=200);

# plt.hist2d(good_WD['Chi0_5'],good_WD['S_N'],bins=200);
